[View in Colaboratory](https://colab.research.google.com/github/josd/eye/blob/master/i2i/roots/observation_prediction_roots.ipynb)

# Transduction from observation to prediction for roots of polynomial

## Introduction

What is [Transduction (machine learning)](https://en.wikipedia.org/wiki/Transduction_(machine_learning%29):

> In logic, statistical inference, and supervised learning, transduction or
transductive inference is reasoning from observed, specific (training) cases
to specific (test) cases. In contrast, induction is reasoning from observed
training cases to general rules, which are then applied to the test cases.
The distinction is most interesting in cases where the predictions of the
transductive model are not achievable by any inductive model. Note that this
is caused by transductive inference on different test sets producing mutually
inconsistent predictions.

What is the Tensor2Tensor [Transformer model](https://github.com/tensorflow/tensor2tensor/blob/master/tensor2tensor/models/transformer.py):

> The Transformer model consists of an encoder and a decoder. Both are stacks
of self-attention layers followed by feed-forward layers. This model yields
good results on a number of problems, especially in NLP and machine translation.
See "Attention Is All You Need" (https://arxiv.org/abs/1706.03762) for the full
description of the model and the results obtained with its early version.

![Transformer model](https://pbs.twimg.com/media/DCKhefrUMAE9stK.jpg)

> The encoder is composed of a stack of N identical layers. Each layer has
two sub-layers. The first is a multi-head self-attention mechanism, and the
second is a simple, positionwise fully connected feed-forward network.
There is a residual connection around each of the two sub-layers, followed by
layer normalization.

> The decoder is also composed of a stack of N identical layers. In addition
to the two sub-layers in each encoder layer, the decoder inserts a third
sub-layer, which performs multi-head attention over the output of the encoder
stack. The self-attention sub-layer in the decoder stack is modified to prevent
positions from attending to subsequent positions.  This masking, combined with
the fact that the output embeddings are offset by one position, ensures that the
predictions for position i can depend only on the known outputs at positions
less than i.

In [0]:
# Preparation

# install tensor2tensor
! pip install -q -U tensor2tensor

# show versions
! pip show tensorflow
! pip show tensor2tensor

# get the needed resources
! curl -O http://josd.github.io/eye/i2i/roots/run.sh
! curl -O http://josd.github.io/eye/i2i/roots/observation_prediction_roots.py
! curl -O http://josd.github.io/eye/i2i/roots/__init__.py
! curl -O http://josd.github.io/eye/i2i/roots/test_roots.observation
! chmod +x run.sh

# clear data and model
% rm -fr /tmp/t2t_data/observation_prediction_roots/
% rm -fr /tmp/t2t_train/observation_prediction_roots/transformer-transformer_small/

# start tensorboard
! curl -O https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -o ngrok-stable-linux-amd64.zip
get_ipython().system_raw('tensorboard --logdir /tmp/t2t_train/observation_prediction_roots/transformer-transformer_small --host 0.0.0.0 --port 6006 &')
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [0]:
# See the observation_prediction_roots problem

! pygmentize -g observation_prediction_roots.py

In [0]:
# See the run.sh script

! pygmentize -g run.sh

In [0]:
# Run the run.sh script

! ./run.sh

In [0]:
# See the transductions

! pygmentize -g test_roots.observation
print("->-")
! pygmentize -g test_roots.prediction

In [0]:
import os

import tensorflow as tf

from tensor2tensor import problems
from tensor2tensor.bin import t2t_decoder  # To register the hparams set
from tensor2tensor.utils import registry
from tensor2tensor.utils import trainer_lib
from tensor2tensor.visualization import attention
from tensor2tensor.visualization import visualization

In [0]:
def call_html():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              "d3": "https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.8/d3.min",
              jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
            },
          });
        </script>
        '''))

In [0]:
# MODEL
CHECKPOINT = os.path.expanduser('/tmp/t2t_train/observation_prediction_roots/transformer-transformer_small')

In [0]:
# HPARAMS
problem_name = 'observation_prediction_roots'
data_dir = os.path.expanduser('/tmp/t2t_data/observation_prediction_roots')
model_name = "transformer"
hparams_set = "transformer_small"

In [0]:
import observation_prediction_roots

visualizer = visualization.AttentionVisualizer(hparams_set, model_name, data_dir, problem_name, beam_size=1)

In [0]:
tf.Variable(0, dtype=tf.int64, trainable=False, name='global_step')

sess = tf.train.MonitoredTrainingSession(
    checkpoint_dir=CHECKPOINT,
    save_summaries_secs=0,
)

In [0]:
input_sentence = "_:POLYNOMIAL <i2i#coefficients> (2 -1 9)."
output_string, inp_text, out_text, att_mats = visualizer.get_vis_data_from_string(sess, input_sentence)
print(output_string)

## Interpreting the Visualizations
- The layers drop down allow you to view the different Transformer layers, 0-indexed of course.
  - Tip: The first layer, last layer and 2nd to last layer are usually the most interpretable.
- The attention dropdown allows you to select different pairs of encoder-decoder attentions:
  - All: Shows all types of attentions together. NOTE: There is no relation between heads of the same color - between the decoder self attention and decoder-encoder attention since they do not share parameters.
  - Input - Input: Shows only the encoder self-attention.
  - Input - Output: Shows the decoder’s attention on the encoder. NOTE: Every decoder layer attends to the final layer of encoder so the visualization will show the attention on the final encoder layer regardless of what layer is selected in the drop down.
  - Output - Output: Shows only the decoder self-attention. NOTE: The visualization might be slightly misleading in the first layer since the text shown is the target of the decoder, the input to the decoder at layer 0 is this text with a GO symbol prepreded.
- The colored squares represent the different attention heads.
  - You can hide or show a given head by clicking on it’s color.
  - Double clicking a color will hide all other colors, double clicking on a color when it’s the only head showing will show all the heads again.
- You can hover over a word to see the individual attention weights for just that position.
  - Hovering over the words on the left will show what that position attended to.
  - Hovering over the words on the right will show what positions attended to it.

In [0]:
call_html()
attention.show(inp_text, out_text, *att_mats)